In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [3]:
df.head()

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [4]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

In [5]:
df["Airline"].value_counts()

KLM(!)                 1
"Swiss Air"            1
(British Airways. )    1
12. Air France         1
<Air France> (12)      1
Name: Airline, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   From_To       5 non-null      object 
 1   FlightNumber  3 non-null      float64
 2   RecentDelays  5 non-null      object 
 3   Airline       5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
imputer = SimpleImputer(missing_values=np.nan,strategy="median",)

In [9]:
df.FlightNumber = df.FlightNumber+10

In [10]:
df.FlightNumber = imputer.fit_transform(df.FlightNumber.values.reshape(-1,1))[:,0]

In [11]:
df.head()

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10055.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10075.0,[],<Air France> (12)
2,londON_StockhOlm,10075.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10095.0,"[67, 32]","""Swiss Air"""


In [12]:
df.FlightNumber = df.FlightNumber.convert_dtypes()

In [13]:
df.head()


,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10055,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10075,[],<Air France> (12)
2,londON_StockhOlm,10075,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10095,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.<br>
3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.base import clone
import string

In [15]:
class Splitter(BaseEstimator,TransformerMixin):
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        return [[string.capwords(name.split("_")[0]),string.capwords(name.split("_")[1])]for name in X]

In [16]:
splitter = Splitter()

In [17]:
from_to_values = splitter.transform(df.From_To.values)

In [18]:
from_to_values

[['London', 'Paris'],
 ['Madrid', 'Milan'],
 ['London', 'Stockholm'],
 ['Budapest', 'Paris'],
 ['Brussels', 'London']]

In [19]:
temp_df = pd.DataFrame(from_to_values,columns=["From","To"])

In [20]:
temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [21]:
df.drop("From_To",axis=1,inplace=True)

In [22]:
df

,FlightNumber,RecentDelays,Airline
0,10055,"[23, 47]",KLM(!)
1,10075,[],<Air France> (12)
2,10075,"[24, 43, 87]",(British Airways. )
3,10075,[13],12. Air France
4,10095,"[67, 32]","""Swiss Air"""


In [23]:
df["From"] = temp_df["From"]
df["To"] = temp_df["To"]

In [24]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10055,"[23, 47]",KLM(!),London,Paris
1,10075,[],<Air France> (12),Madrid,Milan
2,10075,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10095,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [25]:
val = df.RecentDelays.values

In [26]:
val

array([list([23, 47]), list([]), list([24, 43, 87]), list([13]),
       list([67, 32])], dtype=object)

In [27]:
def transform(X):
    for indexx,i in enumerate(X):
        for index,a in enumerate(i):
            if "delay_{}".format(index) not in df.columns:
                df["delay_{}".format(index)] = np.nan
            df.loc[indexx,"delay_{}".format(index)] = a

In [28]:
transform(df.RecentDelays.values)

In [29]:
df.rename(columns={"RecentDelays":"delays"})

,FlightNumber,delays,Airline,From,To,delay_0,delay_1,delay_2
0,10055,"[23, 47]",KLM(!),London,Paris,23.0,47.0,NaN
1,10075,[],<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10075,"[24, 43, 87]",(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,[13],12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10095,"[67, 32]","""Swiss Air""",Brussels,London,67.0,32.0,NaN
